In [3]:
import sys
sys.path.append('../utilities/')

In [4]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import torch
from sentence_transformers import SentenceTransformer
from joblib import dump
from openai import OpenAI
from tqdm import tqdm
from mmd import MMD
import re
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import os
from dotenv import load_dotenv

# **Tools Setup**

In [ ]:
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=API_KEY)

In [4]:
sentence_transformer = SentenceTransformer('all-mpnet-base-v2')

# **DataFrame Reading**

In [5]:
df = pd.read_csv('../data/initial_datasets/dota2_train_labels_translated_sanitized.csv')

In [6]:
sample_df = df.sample(n=1000)

In [19]:
sample_df

,message,label,player,special_token_message,match,translated_message,sanitized_translated_message
185,EZ,NaN,3,[Team0][Player3]: EZ,21,Easy,Easy
846,tal vez la voltee jaja,NaN,4,[Team0][Player4]: tal vez la voltee jaja,70,"""maybe I messed it up haha""",maybe I messed it up haha
1690,gg,NaN,5,[Team1][Player0]: gg,84,gg,gg
1215,SO HARD TO BE HERE,NaN,9,[Team1][Player4]: SO HARD TO BE HERE,12,SO HARD TO BE HERE,SO HARD TO BE HERE
3036,sorry guys,NaN,2,[Team0][Player2]: sorry guys,67,sorry guys,sorry guys
...,...,...,...,...,...,...,...
624,haha,NaN,5,[Team1][Player0]: haha,18,haha,haha
3081,АХах,NaN,7,[Team1][Player2]: АХах,62,"""Ahah""",Ahah
479,ang tanga mo,x,5,[Team1][Player0]: ang tanga mo,63,"""you're so stupid""",you're so s*****
2410,АААААААААА,NaN,5,[Team1][Player0]: АААААААААА,93,"""AAAAAAAAA""",AAAAAAAAA


In [24]:
sample_df['label'] = sample_df['label'].fillna(0)
sample_df['label'] = sample_df['label'].replace({'x': 1})
sample_df['label'] = sample_df['label'].astype(int)

# **Aggregate Stats**

In [42]:
sample_df['phrase_len'] = sample_df['translated_message'].str.len()
avg_len = sample_df['phrase_len'].mean()
std_len = sample_df['phrase_len'].std()

## **Word Frequencies**

In [43]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sample_df['translated_message'])
counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
total_counts = counts.sum().sort_values(ascending=False)

In [44]:
total_counts = total_counts.reset_index()
total_counts.columns = ['text', 'count']
total_counts['freq'] = total_counts['count'] / total_counts['count'].sum()

## **Non-toxic data Frequency**

In [45]:
non_toxic_df = sample_df[sample_df['label'] == 0]
X = vectorizer.fit_transform(non_toxic_df['translated_message'])
counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
positive_counts = counts.sum().sort_values(ascending=False)

## **Toxic data Frequency**

In [51]:
toxic_df = sample_df[sample_df['label'] == 1]
X = vectorizer.fit_transform(toxic_df['translated_message'])
counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
negative_counts = counts.sum().sort_values(ascending=False)

In [52]:
negative_counts

you         12
fucking      6
bitch        6
fuck         4
go           4
            ..
gg           1
game         1
fucktard     1
fuckface     1
zero         1
Length: 99, dtype: int64

## **Compare Across Both**

In [53]:
positive_counts = positive_counts.reset_index()
positive_counts.columns = ['word', 'freq']
negative_counts = negative_counts.reset_index()
negative_counts.columns = ['word', 'freq']

In [54]:
positive_counts['freq'] = positive_counts['freq'] / positive_counts['freq'].sum()
negative_counts['freq'] = negative_counts['freq'] / negative_counts['freq'].sum()

In [ ]:
positive_counts['rank'] = positive_counts.index+1
negative_counts['rank'] = negative_counts.index+1

In [77]:
merged_df = pd.merge(positive_counts, negative_counts, how='outer', on='rank')
merged_df = merged_df.drop(columns=['rank'])
merged_df = merged_df.rename(columns={'word_x': 'non_toxic_word', 'freq_x': 'non_toxic_freq', 'word_y': 'toxic_word', 'freq_y': 'toxic_freq'})

In [78]:
merged_df = merged_df.iloc[:99]

In [79]:
merged_df

,non_toxic_word,non_toxic_freq,toxic_word,toxic_freq
0,you,0.041032,you,0.082759
1,the,0.024535,fucking,0.041379
2,gg,0.021574,bitch,0.041379
3,to,0.017343,fuck,0.027586
4,easy,0.014382,go,0.027586
...,...,...,...,...
94,lost,0.002115,gg,0.006897
95,wait,0.002115,game,0.006897
96,didn,0.001692,fucktard,0.006897
97,sniper,0.001692,fuckface,0.006897


# **Chat Generation**

## **Instruction Based Prompting**

In [80]:
common_words = "\n".join([
    f"{row['non_toxic_word']}: {row['non_toxic_freq']};  {row['toxic_word']}: {row['toxic_freq']}"
    for _, row in merged_df.iterrows()
])

In [81]:
instruction = (
    "You are a data generator tasked with creating realistic DOTA 2 chat messages. "
    "These chat messages should be labeled according to their sentiment: toxic or non-toxic.\n"
    "Base the style on typical video game chat messages — include informal internet language, typos, and abbreviations\n"
    "You will be given statistics about the distribution, including average chat length, standard deviation, and most common words associated with each label and their frequency.\n"
    "Generate exactly 10 realistic DOTA 2 chat messages, one per line.\n"
    "Each line should follow this format: the chat message in double quotes, followed by a space and then the label (0 for toxic, 1 for non-toxic).\n"
    "No extra formatting — just plain text output, one line per comment.\n"
    "Here is the format:\n"
    "\"gg dawg\" 0\n"
    "\"I hate u bitch\" 1"
)
input = (
    f"Here is the average length of all the comments: {avg_len}. "
    f"Here is the standard deviation of the length for all the comments: {std_len}.\n"
    f"Here are the most frequent words (format: word: frequency (sentiment)):\n\n{common_words}\n"
    f"Now, generate the 10 new comments below:"
)

In [87]:
res = []
for i in range(50):
    response = client.responses.create(
        model="gpt-4o",
        instructions=instruction,
        input=input
    )
    res.append(response.output_text)

In [88]:
labels = []
sentences = []
for i in range(50):
    for word in res[i].split("\n"):
        match = re.match(r'"(.*?)"\s*(-?\d+)', word)
        if match:
            quoted = match.group(1)      
            label = match.group(2)       
            sentences.append(quoted)
            labels.append(int(label))

In [89]:
generated_df = pd.DataFrame({
    'sentences': sentences,
    'labels': labels
})

In [86]:
old_df = generated_df

In [91]:
generated_df = pd.concat([old_df, generated_df], axis=0)

In [92]:
generated_df.to_csv('../data/generated/dota2/privacy_protected/summary_stats_dota2_gen.csv', index=False)